In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Dense, TimeDistributed, LSTM, InputLayer, 
    Flatten, GRU, Bidirectional, Activation, Reshape, concatenate)
from tensorflow import keras


In [27]:
# surrogate data
n_channels = 32
n_timepoints = 3
n_samples = 100
n_dipoles = 1234

X = np.random.randn(n_samples, n_timepoints, n_channels)
y = x = np.random.randn(n_samples, n_timepoints, n_dipoles)

In [29]:
model_s.predict(X).shape

(100, 3, 1234)

In [28]:
# NEW MODEL
n_dense_units = 100
n_lstm_units = 25
activation_function = 'relu'
dropout = 0.2

inputs = keras.Input(shape=(n_timepoints, n_channels), name='Input')
fc1 = TimeDistributed(Dense(n_dense_units), name='FC1')(inputs)
fc2 = TimeDistributed(Dense(n_dense_units), name='FC2')(fc1)

flat = Flatten()(fc2)

last_fc = Dense(int(n_dipoles*n_timepoints), name='FC_Out')(flat)
out = Reshape((n_timepoints, n_dipoles))(last_fc)
model_s = keras.Model(inputs=inputs, outputs=out)

model_s.summary()

# # SINGLE TIME FRAME PATH
# fc1 = TimeDistributed(Dense(n_dense_units), name='FC1')(inputs)
# fc2 = TimeDistributed(Dense(n_dipoles), name='FC2')(fc1)

# model_s = keras.Model(inputs=inputs, outputs=fc2, name='single_time_ frame_model')
# # model_s.summary()

# # # MULTI TIME FRAME PATH
# lstm1 = Bidirectional(LSTM(n_lstm_units, return_sequences=True, 
#     input_shape=(None, n_dense_units), dropout=dropout, 
#     activation=activation_function), name='LSTM1')(fc1)

# concat = concatenate([lstm1, fc2], name='Concat')

# lstm2 = Bidirectional(LSTM(n_lstm_units, return_sequences=True, 
#     input_shape=(None, n_dense_units), dropout=dropout, 
#     activation=activation_function), name='LSTM2')(concat)

# output = TimeDistributed(Dense(n_dipoles), name='FC_Out')(lstm2)
# model_m = keras.Model(inputs=inputs, outputs=output, name='multi_time_frame_model')

# model_m.summary()
# model_m.compile(loss='mse')
# model_m.fit(X, y)

# assert model_m.predict(X).shape == y.shape, 'shape mismatch'

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 3, 32)]           0         
_________________________________________________________________
FC1 (TimeDistributed)        (None, 3, 100)            3300      
_________________________________________________________________
FC2 (TimeDistributed)        (None, 3, 100)            10100     
_________________________________________________________________
flatten_7 (Flatten)          (None, 300)               0         
_________________________________________________________________
FC_Out (Dense)               (None, 3702)              1114302   
_________________________________________________________________
reshape_5 (Reshape)          (None, 3, 1234)           0         
Total params: 1,127,702
Trainable params: 1,127,702
Non-trainable params: 0
_________________________________________________

In [ ]:
model_m.predict(np.random.randn(1, 11, 32)).

In [ ]:
# OLD MODEL
model = tf.keras.Sequential()
tf.keras.backend.set_image_data_format('channels_last')
input_shape = (None, n_channels)
model.add(InputLayer(input_shape=input_shape))

# LSTM layers
for _ in range(1):
    model.add(Bidirectional(LSTM(20, 
        return_sequences=True, input_shape=input_shape, 
        dropout=0.2, activation='relu')))

# Hidden Dense layer(s):
for _ in range(1):
    model.add(Dense(100, 
        activation='relu'))

# Final For-each layer:
model.add(TimeDistributed(
    Dense(n_dipoles, activation='linear'))
)

model.build(input_shape=input_shape)

assert model.predict(X).shape == y.shape, 'shape mismatch'